In [1]:
from model.main_model import Classifier
from data.dataset import RuStoreDataset

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd

/Users/daniilogorodnikov/PycharmProjects/RuStore-type/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/daniilogorodnikov/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/7710840340a098cfb869c4f65e87cf2b1b70caca/modeling_hf_nomic_bert.py:1634: UserWarning: Install Nomic's megablocks fork for better speed: `pip install git+https://github.com/nomic-ai/megablocks.git`
  warnings.warn("Install Nomic's megablocks fork for better speed: " +


In [2]:
train = pd.read_csv("data/train.tsv", sep='\t').dropna()

dataset = RuStoreDataset(train)
batch_size = 2 # Adjust as needed
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [3]:
input_dim = 2304  # As per the model
num_classes = 394  # As per the model

model = Classifier(input_dim=input_dim, num_classes=num_classes)
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
model.to(device)

criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)  # Adjust learning rate as needed
# Optional: Scheduler
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Training parameters
num_epochs = 10  # Adjust as needed

In [4]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
from tqdm import tqdm

In [6]:

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(train_loader):
        inputs = inputs.to(device)  # Inputs shape: (bs, 3, 768) or similar, but model reshapes
        labels = labels.to(device)  # Labels: (bs,)

        optimizer.zero_grad()

        # Get logits (without softmax, for loss)
        logits = model.get_logits(inputs)

        # Compute loss
        loss = criterion(logits, labels)

        # Backpropagation
        loss.backward()

        optimizer.step()

        # Track loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(logits.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(correct)
        print(running_loss)

    # Optional: Step the scheduler
    # scheduler.step()

    # Epoch statistics
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / len(train_loader)

    print(f"epoch_loss: {epoch_loss}, epoch_acc: {epoch_acc}")
    torch.save(model.state_dict(), f"models/{epoch}.pth")

  0%|          | 0/26747 [00:00<?, ?it/s]/Users/daniilogorodnikov/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/7710840340a098cfb869c4f65e87cf2b1b70caca/modeling_hf_nomic_bert.py:1634: UserWarning: Install Nomic's megablocks fork for better speed: `pip install git+https://github.com/nomic-ai/megablocks.git`
  warnings.warn("Install Nomic's megablocks fork for better speed: " +
/Users/daniilogorodnikov/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/7710840340a098cfb869c4f65e87cf2b1b70caca/modeling_hf_nomic_bert.py:1634: UserWarning: Install Nomic's megablocks fork for better speed: `pip install git+https://github.com/nomic-ai/megablocks.git`
  warnings.warn("Install Nomic's megablocks fork for better speed: " +
/Users/daniilogorodnikov/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/7710840340a098cfb869c4f65e87cf2b1b70caca/modeling_hf_nomic_bert.py:1634: UserWarning: Install Nomic's megablocks fork for better 

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/daniilogorodnikov/PycharmProjects/RuStore-type/venv/lib/python3.13/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/Users/daniilogorodnikov/PycharmProjects/RuStore-type/venv/lib/python3.13/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/Users/daniilogorodnikov/PycharmProjects/RuStore-type/data/dataset.py", line 24, in __getitem__
    app_name_emb = embedded([X['app_name']])
  File "/Users/daniilogorodnikov/PycharmProjects/RuStore-type/model/embedding.py", line 18, in embedded
    model_output = model(**encoded_input)
  File "/Users/daniilogorodnikov/PycharmProjects/RuStore-type/venv/lib/python3.13/site-packages/torch/nn/modules/module.py", 